In [ ]:
# Testing Cell
from aviary.interface.methods_for_level2 import AviaryGroup
from aviary.utils.doctape import glue_class_options

current_glued_vars = []

glue_class_options(AviaryGroup, current_glued_vars, md_code=True)

# Models with External Subsystems

In level 2, we have given simple examples of defining external subsystems in {glue:md}`mission_info`. The subsystems that we gave were all dummy subsystems and are not really used in simulation. Assume you have an external subsystem and you want to use it. Let us show you how to achieve this goal.

## Installation of Aviary examples

The Aviary team has provided a few external subsystems for you to use.
These are included in the {glue:md}`aviary/examples/external_subsystems` directory.
We'll now discuss them here and show you how to use them.

In [ ]:
# Testing Cell
import aviary.api as av
from aviary.api import Aircraft
from aviary.utils.doctape import get_variable_name, glue_variable
from aviary.utils.functions import get_model, get_path

path = get_path('examples/external_subsystems').relative_to(av.top_dir.parent)
glue_variable(path, md_code=True)
path = get_path('examples/external_subsystems/battery')
glue_variable(path.stem, md_code=True)
path = get_path('examples/external_subsystems/custom_mass')
glue_variable(path.stem, md_code=True)

glue_variable(get_variable_name(Aircraft.Wing.MASS), md_code=True)
glue_variable(get_variable_name(Aircraft.HorizontalTail.MASS), md_code=True)

aircraft_csv_file = 'aircraft_for_bench_FwFm.csv'
get_model(aircraft_csv_file)
glue_variable(aircraft_csv_file, md_code=True)

## Adding Subsystems

Currently, there are a couple of examples: {glue:md}`battery` and {glue:md}`custom_mass`. Let us take a look at {glue:md}`custom_mass` first. As shown in this example, this is a simplified example of a component that computes a weight for the wing and horizontal tail. It does not provide realistic computations but rough estimates to {glue:md}`Aircraft.Wing.MASS` and {glue:md}`Aircraft.HorizontalTail.MASS`. When this external subsystem is added to your pre-mission phase, Aviary will compute these weights in its core subsystem as usual, but then the wing mass and tail mass values will be overridden by this external subsytem.

In [level 2](onboarding_level2), we have briefly covered how to add external subsystems in {glue:md}`mission_info`. Alternatively, external subsystems (and any other new keys) can be added after a {glue:md}`mission_info` is loaded. Let us see how it works using the {glue:md}`aircraft_for_bench_FwFm.csv` model. First, we import this particular external subsystem.

Then add this external subsystem to `pre_mission`.
That is all you need to do in addition to our traditional level 2 examples. Here is the complete run script,

In [ ]:
# Testing Cell
import aviary.api as av
from aviary.api import Aircraft
from aviary.utils.doctape import check_value, glue_variable

Aircraft.Wing.MASS
Aircraft.HorizontalTail.MASS
check_value(av.EquationsOfMotion.HEIGHT_ENERGY.value, 'height_energy')
glue_variable(av.EquationsOfMotion.HEIGHT_ENERGY.value, md_code=False)

In [ ]:
from copy import deepcopy

import aviary.api as av
from aviary.api import Aircraft
from aviary.examples.external_subsystems.custom_mass.custom_mass_builder import WingMassBuilder

# Max iterations set to 1 to reduce runtime of example
max_iter = 1
phase_info = deepcopy(av.default_height_energy_phase_info)

prob = av.AviaryProblem()

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(
    'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv',
    phase_info,
)

prob.load_external_subsystems([WingMassBuilder(name='wing_external')])

prob.check_and_preprocess_inputs()

prob.build_model()

prob.add_driver('SLSQP', max_iter)

prob.add_design_variables()

prob.add_objective()

prob.setup()

prob.run_aviary_problem(suppress_solver_print=True)

print('Engine Mass', prob.get_val(av.Aircraft.Engine.MASS))
print('Wing Mass', prob.get_val(av.Aircraft.Wing.MASS))
print('Horizontal Tail Mass', prob.get_val(av.Aircraft.HorizontalTail.MASS))

Ignore the intermediate warning messages and you see the outputs at the end.
Since this is a {glue:md}`height_energy` mission and no objective is provided, we know that the objective is `fuel_burned`.

To see the outputs without external subsystem add-on, let us comment out the line that adds the wing weight builder and run the modified script:

In [ ]:
# Max iterations set to 1 to reduce runtime of example
max_iter = 1
prob = av.AviaryProblem()

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs(
    'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv',
    av.default_height_energy_phase_info,
)

prob.load_external_subsystems([WingMassBuilder(name='wing_external')])

prob.check_and_preprocess_inputs()

prob.build_model()

prob.add_driver('SLSQP', max_iter)

prob.add_design_variables()

prob.add_objective()

prob.setup()

prob.run_aviary_problem(suppress_solver_print=True)

print('Engine Mass', prob.get_val(Aircraft.Engine.MASS))
print('Wing Mass', prob.get_val(Aircraft.Wing.MASS))
print('Horizontal Tail Mass', prob.get_val(Aircraft.HorizontalTail.MASS))

As we see, the engine mass is not altered but wing mass and tail mass are changed dramatically. This is not surprising because our {glue:md}`custom_mass` subsystem is quite simple. Later on, we will show you a more realistic wing weight external subsystem.

In [ ]:
# Testing Cell
import inspect

from aviary.interface.methods_for_level2 import AviaryProblem
from aviary.utils.doctape import check_contains, glue_function_arguments

imported_functions = {k for k, v in inspect.getmembers(AviaryProblem, inspect.isfunction)}
check_contains(
    [
        '__init__',
        'load_inputs',
        'build_model',
        'add_driver',
        'add_design_variables',
        'add_objective',
        'setup',
        'set_initial_guesses',
        'run_aviary_problem',
    ],
    imported_functions,
)

# Retrieve all arguments from add_driver()
glue_function_arguments(AviaryProblem.add_driver, current_glued_vars, md_code=True)

In [ ]:
# Testing Cell
from aviary.interface.methods_for_level2 import AviaryProblem
from aviary.models.missions.height_energy_default import phase_info
from aviary.utils.doctape import glue_class_functions, glue_variable

# Get all functions of class AviaryProblem
glue_class_functions(AviaryProblem, [], md_code=False)

# Retrieve all top-level keys of phase_info
top_level_keys_list = [k for k in phase_info.keys() if k not in ('pre_mission', 'post_mission')]
for key in top_level_keys_list:
    glue_variable(key, md_code=False)

## Adding battery subsystem

In the above example, there is no new Aviary variable added to Aviary and the external subsystem is added to pre-mission only. So, the subsystem is not very involved. We will see a more complicated example now. Before we move on, let us recall the steps in Aviary model building:

- **{glue:md}`__init__()`**
- **{glue:md}`load_inputs()`**
- **{glue:md}`load_external_subsystems()`** (optional)
- **{glue:md}`build_model()`**
- **{glue:md}`check_and_preprocess_inputs()`**
- {glue:md}`add_driver()`
- **{glue:md}`add_design_variables()`**
- **{glue:md}`add_objective()`**
- {glue:md}`setup()`
- {glue:md}`run_aviary_problem()`

The steps in bold are related specifically to subsystems. So, almost all of the steps involve subsystems. As long as your external subsystem is built based on the guidelines, Aviary will take care of your subsystem.

The next example is the [battery subsystem](https://github.com/OpenMDAO/Aviary/blob/main/aviary/docs/user_guide/battery_subsystem_example). The battery subsystem provides methods to define the battery subsystem's states, design variables, fixed values, initial guesses, and mass names. It also provides methods to build OpenMDAO systems for the pre-mission and mission computations of the subsystem, to get the constraints for the subsystem, and to preprocess the inputs for the subsystem. This subsystem has its own set of variables. We will build an Aviary model with full phases (namely, {glue:md}`climb`, {glue:md}`cruise` and {glue:md}`descent`) and maximize the final total mass: {glue:md}`Dynamic.Vehicle.MASS`.

In [ ]:
# Testing Cell
from aviary.api import Dynamic
from aviary.examples.external_subsystems.battery.battery_builder import BatteryBuilder
from aviary.utils.doctape import glue_variable

glue_variable(BatteryBuilder.__name__, md_code=True)

We also need {glue:md}`BatteryBuilder` along with battery related aircraft variables and build a new battery object.

In [ ]:
from aviary.examples.external_subsystems.battery.battery_builder import BatteryBuilder
from aviary.examples.external_subsystems.battery.battery_variable_meta_data import ExtendedMetaData
from aviary.examples.external_subsystems.battery.battery_variables import Aircraft

battery_builder = BatteryBuilder(include_constraints=False)

Start an Aviary problem and load in an aircraft input deck as well as our new battery builder object:

In [ ]:
prob = av.AviaryProblem()

prob.load_inputs(
    'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv',
    phase_info,
    meta_data=ExtendedMetaData,
)

prob.load_external_subsystems([battery_builder])

prob.check_and_preprocess_inputs()

Since this example contains new variables in the aircraft hierarchy, the metadata for those variables was added to an extended metadata dictionary. We need to pass that into load_inputs so that it can load susbsystem-specific inputs from the csv file.

The inputs are then checked by calling check_and_preprocess_inputs.

In the battery subsystem, the type of battery cell we use is `18650`. This type of battery information is set in `preprocess_inputs()` within `examples/external_subsystems/batterybattery_builder.py`.

### Checking in the setup function call

The OpenMDAO Function {glue:md}`setup()` can have an argument `check` with default value `None` (see [here](https://openmdao.org/newdocs/versions/latest/features/core_features/running_your_models/setup.html)). If we set it to `True`, it will cause a default set of checks to be run. So, instead of a simple {glue:md}`setup()` call, let us call it with `check=True`.

The following are a few check points printed on the command line:

```
INFO: checking out_of_order
INFO: checking system
INFO: checking solvers
INFO: checking dup_inputs
INFO: checking missing_recorders
```


In [ ]:
prob.build_model()

max_iter = 1
prob.add_driver('SLSQP', max_iter)

prob.add_design_variables()

prob.add_objective('mass')

prob.setup(check=True)

prob.set_initial_guesses()

prob.run_aviary_problem()

# user defined outputs
print('Battery MASS', prob.get_val(Aircraft.Battery.MASS))
print('Cell Max', prob.get_val(Aircraft.Battery.Cell.MASS))
masses_descent = prob.get_val('traj.descent.timeseries.mass', units='kg')
print(f'Final Descent Mass: {masses_descent[-1]}')

print('done')

## More on outputs

We are done with our model. For our current example, let us add a few more lines after the aviary run:

In [ ]:
print('Battery MASS', prob.get_val(Aircraft.Battery.MASS, units='lbm'))
print('Cell Max', prob.get_val(Aircraft.Battery.Cell.MASS))

In [ ]:
# Testing Cell
from aviary.utils.doctape import get_variable_name, glue_variable

glue_variable(get_variable_name(Dynamic.Vehicle.MASS), md_code=True)

Since our objective is `mass`, we want to print the value of {glue:md}`Dynamic.Vehicle.MASS`. Remember, we have imported Dynamic from aviary.variable_info.variables for this purpose.

So, we have to print the final mass in a different way. Keep in mind that we have three phases in the mission and that final mass is our objective. So, we can get the final mass of the descent phase instead. Let us try this approach. Let us comment out the print statement of final mass (and the import of Dynamic), then add the following lines:

In [ ]:
masses_descent = prob.get_val('traj.descent.timeseries.mass', units='kg')
print(f'Final Descent Mass: {masses_descent[-1]}')

In [ ]:
# Testing Cell
from openmdao.utils.assert_utils import assert_near_equal

from aviary.examples.external_subsystems.battery.battery_builder import BatteryBuilder
from aviary.examples.external_subsystems.battery.battery_variable_meta_data import ExtendedMetaData
from aviary.examples.external_subsystems.battery.battery_variables import Dynamic

battery_builder = BatteryBuilder(include_constraints=False)

prob = av.AviaryProblem()

prob.load_inputs(
    'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv',
    phase_info,
    meta_data=ExtendedMetaData,
)

prob.load_external_subsystems([battery_builder])

prob.check_and_preprocess_inputs()

prob.build_model()

max_iter = 1
prob.add_driver('SLSQP', max_iter)

prob.add_design_variables()

# prob.add_objective('mass')
prob.model.add_objective(f'traj.climb.states:{Dynamic.Battery.STATE_OF_CHARGE}', index=-1, ref=-1)

prob.setup(check=True)

prob.set_initial_guesses()

prob.run_aviary_problem()


soc_cruise = prob.get_val('traj.climb.timeseries.dynamic:battery:state_of_charge')
state_of_charge = soc_cruise[-1]
expected_soc = 0.91333333
assert_near_equal(state_of_charge[0], expected_soc, 1e-5)
glue_variable('expected_soc', '[' + str(expected_soc) + ']', md_code=False)

# Level 3

Level 3 represents the highest level of user control and customization in Aviary's user interface.
At this level, users have full access to Python and OpenMDAO methods that Aviary calls.
They can use the complete set of Aviary's methods, functionalities, and classes to construct and fine-tune their aircraft models.
Level 3 enables users to have supreme control over every aspect of the model, including subsystems, connections, and advanced optimization techniques.

Level 3 is the most complex but specific methods defined at this level are used in levels 1 and 2, hopefully reducing user activation energy when learning more about Aviary.
This progressive approach helps users gradually enhance their analysis capabilities and adapt to more complex modeling requirements as they gain proficiency and experience.

## More on objectives

Now, let us change our objective to battery state of charge after the climb phase. So, comment out `prob.add_objective('mass')` and add the following line right after:

```
prob.model.add_objective(
    f'traj.climb.states:{Dynamic.Battery.STATE_OF_CHARGE}', index=-1, ref=-1)
```

In the above, `index=-1` means the end of climb phase and `ref=-1` means that we want to maximize the state of charge at the end of climb phase. Once again, we are unable to print battery state of charge as we did with battery mass and battery cell mass. We will use the same approach to get mass. In fact, we have prepared for this purpose by setting up time series of climb and cruise phases as well. All we need to do is to add the following lines:

```
soc_cruise = prob.get_val(
    'traj.climb.timeseries.dynamic:battery:state_of_charge')
print(f'State of Charge: {soc_cruise[-1]}')
```

Now you get a new output:

State of Charge: {glue:md}`expected_soc`


## The check_partials function

In order to make sure that your model computes all the derivatives correctly, OpenMDAO provides a [method called `check_partials`](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_derivatives/basic_check_partials.html) which checks partial derivatives comprehensively for all Components in your model.
You should check your partial derivatives before integrating your external subsystem.
This is a good practice to ensure that your model is working correctly and can be used in an optimization context.

## Adding an OpenAeroStruct wingbox external subsystem

[OpenAeroStruct](https://github.com/mdolab/OpenAerostruct/) (OAS) is a lightweight tool that performs aerostructural optimization using OpenMDAO. This is an example that shows you how to use an existing external package with Aviary.

### Installation of OpenAeroStruct

We would like to have easy access to the examples and source code. So we install OpenAeroStruct by cloning the OpenAeroStruct repository. We show you how to do the installation on Linux. Assume you want to install it at `~/$USER/workspace`. Do the following:

```
cd ~/$USER/workspace
git clone https://github.com/mdolab/OpenAeroStruct.git
~/$USER/workspace/OpenAeroStruct
pip install -e .
```

If everything runs smoothly, you should see something like:

```
Successfully installed openaerostruct
```

Most of the packages that OpenAeroStruct depends on are installed already (see [here](https://mdolab-openaerostruct.readthedocs-hosted.com/en/latest/installation.html)). For our example, we need [ambiance](https://pypi.org/project/ambiance/) and an optional package: [OpenVSP](http://openvsp.org/).

To install `ambiance`, do the following:

```
pip install ambiance
```

You should see something like:

```
Installing collected packages: ambiance
Successfully installed ambiance-1.3.1
```

```{note}
You must ensure that the Python version in your environment matches the Python used to compile OpenVSP. You must [install OpenVSP](https://openvsp.org/wiki/doku.php?id=install) on your Linux box yourself.
```

To check your installation of OpenVSP is successful, please run

```
(av1)$ python openaerostruct/tests/test_vsp_aero_analysis.py
```

Windows users should visit [OpenVSP](http://openvsp.org/download.php) and follow the instruction there.

### Understanding the OpenAeroStruct Example

The OpenAeroStruct example is explained in detail in [Using Aviary and OpenAeroStruct Together](../examples/OAS_subsystem).

In [ ]:
# Testing Cell
from aviary.examples.external_subsystems.OAS_mass.OAS_wing_mass_builder import (
    OASWingMassBuilder,
)
from aviary.utils.doctape import glue_variable

glue_variable(OASWingMassBuilder.__name__, md_code=True)

### Running the OpenAeroStruct Example

We are ready to run this example. First, we create an {glue:md}`OASWingMassBuilder` instance.

In [ ]:
import numpy as np
import openmdao.api as om

import aviary.api as av
from aviary.examples.external_subsystems.OAS_mass.OAS_wing_mass_builder import OASWingMassBuilder

wing_weight_builder = OASWingMassBuilder()

Let's add a few phases in the mission.

In [ ]:
# Load the phase_info and other common setup tasks
phase_info = {
    'climb_1': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 5,
            'order': 3,
            'distance_solve_segments': False,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.2, 'unitless'),
            'mach_final': (0.72, 'unitless'),
            'mach_bounds': ((0.18, 0.74), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (0.0, 'ft'),
            'altitude_final': (32000.0, 'ft'),
            'altitude_bounds': ((0.0, 34000.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'time_initial_bounds': ((0.0, 0.0), 'min'),
            'time_duration_bounds': ((64.0, 192.0), 'min'),
        },
        'initial_guesses': {'time': ([0, 128], 'min')},
    },
    'climb_2': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 5,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.72, 'unitless'),
            'mach_final': (0.72, 'unitless'),
            'mach_bounds': ((0.7, 0.74), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (32000.0, 'ft'),
            'altitude_final': (34000.0, 'ft'),
            'altitude_bounds': ((23000.0, 38000.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'time_initial_bounds': ((64.0, 192.0), 'min'),
            'time_duration_bounds': ((56.5, 169.5), 'min'),
        },
        'initial_guesses': {'time': ([128, 113], 'min')},
    },
    'descent_1': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 5,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.72, 'unitless'),
            'mach_final': (0.36, 'unitless'),
            'mach_bounds': ((0.34, 0.74), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (34000.0, 'ft'),
            'altitude_final': (500.0, 'ft'),
            'altitude_bounds': ((0.0, 38000.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'time_initial_bounds': ((120.5, 361.5), 'min'),
            'time_duration_bounds': ((29.0, 87.0), 'min'),
        },
        'initial_guesses': {'time': ([241, 58], 'min')},
    },
    'post_mission': {
        'include_landing': False,
        'constrain_range': True,
        'target_range': (1800.0, 'nmi'),
    },
}

phase_info['pre_mission'] = {'include_takeoff': False, 'optimize_mass': True}

We can now create an Aviary problem, load in an aircraft input deck, our external subsystem, and do routine input checks:

In [ ]:
aircraft_definition_file = 'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv'
make_plots = False
max_iter = 0
optimizer = 'SNOPT'

prob = av.AviaryProblem()

prob.load_inputs(aircraft_definition_file, phase_info)
prob.load_external_subsystems([wing_weight_builder])
prob.check_and_preprocess_inputs()
prob.build_model()

Next we select the driver and call setup on the problem:

In [ ]:
driver = prob.driver = om.pyOptSparseDriver()
driver.options['optimizer'] = optimizer
driver.declare_coloring()
driver.opt_settings['Major iterations limit'] = max_iter
driver.opt_settings['Major optimality tolerance'] = 1e-4
driver.opt_settings['Major feasibility tolerance'] = 1e-5
driver.opt_settings['iSumm'] = 6

prob.add_design_variables()
prob.add_objective()
prob.setup()

Now we need to set some OpenAeroStruct-specific parameters before running Aviary:

In [ ]:
OAS_sys = 'pre_mission.wing_mass.aerostructures.'
# fmt: off
prob.set_val(
    OAS_sys + 'box_upper_x',
    np.array(
        [
            0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24,
            0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39,
            0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54,
            0.55, 0.56, 0.57, 0.58, 0.59, 0.6,
        ]
    ),
    units='unitless',
)

prob.set_val(
    OAS_sys + 'box_lower_x',
    np.array(
        [
            0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24,
            0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39,
            0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54,
            0.55, 0.56, 0.57, 0.58, 0.59, 0.6,
        ]
    ),
    units='unitless',
)

prob.set_val(
    OAS_sys + 'box_upper_y',
    np.array(
        [
            0.0447, 0.046, 0.0472, 0.0484, 0.0495, 0.0505, 0.0514, 0.0523, 0.0531, 0.0538, 0.0545,
            0.0551, 0.0557, 0.0563, 0.0568, 0.0573, 0.0577, 0.0581, 0.0585, 0.0588, 0.0591, 0.0593,
            0.0595, 0.0597, 0.0599, 0.06, 0.0601, 0.0602, 0.0602, 0.0602, 0.0602, 0.0602, 0.0601,
            0.06, 0.0599, 0.0598, 0.0596, 0.0594, 0.0592, 0.0589, 0.0586, 0.0583, 0.058, 0.0576,
            0.0572, 0.0568, 0.0563, 0.0558, 0.0553, 0.0547, 0.0541,
        ]
    ),
    units='unitless',
)

prob.set_val(
    OAS_sys + 'box_lower_y',
    np.array(
        [
            -0.0447, -0.046, -0.0473, -0.0485, -0.0496, -0.0506, -0.0515, -0.0524, -0.0532, -0.054,
            -0.0547, -0.0554, -0.056, -0.0565, -0.057, -0.0575, -0.0579, -0.0583, -0.0586, -0.0589,
            -0.0592, -0.0594, -0.0595, -0.0596, -0.0597, -0.0598, -0.0598, -0.0598, -0.0598,
            -0.0597, -0.0596, -0.0594, -0.0592, -0.0589, -0.0586, -0.0582, -0.0578, -0.0573,
            -0.0567, -0.0561, -0.0554, -0.0546, -0.0538, -0.0529, -0.0519, -0.0509, -0.0497, 
            -0.0485, -0.0472, -0.0458, -0.0444,
        ]
    ),
    units='unitless',
)
# fmt: on

prob.set_val(OAS_sys + 'twist_cp', np.array([-6.0, -6.0, -4.0, 0.0]), units='deg')
prob.set_val(OAS_sys + 'spar_thickness_cp', np.array([0.004, 0.005, 0.008, 0.01]), units='m')
prob.set_val(OAS_sys + 'skin_thickness_cp', np.array([0.005, 0.01, 0.015, 0.025]), units='m')
prob.set_val(OAS_sys + 't_over_c_cp', np.array([0.08, 0.08, 0.10, 0.08]), units='unitless')
prob.set_val(OAS_sys + 'airfoil_t_over_c', 0.12, units='unitless')
prob.set_val(OAS_sys + 'fuel', 40044.0, units='lbm')
prob.set_val(OAS_sys + 'fuel_reserve', 3000.0, units='lbm')
prob.set_val(OAS_sys + 'CD0', 0.0078, units='unitless')
prob.set_val(OAS_sys + 'cruise_Mach', 0.785, units='unitless')
prob.set_val(OAS_sys + 'cruise_altitude', 11303.682962301647, units='m')
prob.set_val(OAS_sys + 'cruise_range', 3500, units='nmi')
prob.set_val(OAS_sys + 'cruise_SFC', 0.53 / 3600, units='1/s')
prob.set_val(OAS_sys + 'engine_mass', 7400, units='lbm')
prob.set_val(OAS_sys + 'engine_location', np.array([25, -10.0, 0.0]), units='m')

We are now ready to run Aviary on this model.

In [ ]:
prob.run_aviary_problem('oas_solution.db', run_driver=False, make_plots=False)

Note that there are multiple numbers of optimization loops that are output from this run even though we have set {glue:md}`max_iter` to 0.
This is because OpenAeroStruct has an optimization process internally. In order to shorten the runtime, we have set `run_driver = False`. This means that we will not run optimization but [run model](https://openmdao.org/newdocs/versions/latest/features/core_features/running_your_models/run_model.html). 

Finally, we print the newly computed wing mass:

In [ ]:
print('wing mass = ', prob.model.get_val(av.Aircraft.Wing.MASS, units='lbm'))

The result is comparable to the output without OpenAeroStruct external subsystem.